In [15]:
from itertools import combinations 
from random import shuffle 
import numpy as np 

In [16]:
class Card:
    
    def __repr__( self ):
        return '{}{}'.format( self.display_rank, self.display_suit )
    
    def __init__( self, suit, rank ):
        display_suits = [ 's', 'h', 'c', 'd' ]
        display_ranks = [ 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K' ]
        vals = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10 ]
        map_suits = dict(zip(range(4),display_suits))
        map_ranks = dict(zip(range(13),display_ranks))
        map_vals = dict(zip(range(13),vals))

        self.suit = suit
        self.rank = rank 
        self.display_rank = map_ranks[ self.rank ]
        self.display_suit = map_suits[ self.suit ]
        self.value = map_vals[ self.rank ]
        
class Deck:
    def __init__( self ):
        self.cards = []
        for s in range( 4 ):
            for r in range( 13 ):
                self.cards.append(Card(s, r))
                
    def shuffle( self ):
        shuffle( self.cards )
        
    def draw( self, n ):
        result = []
        for i in range( n or 1 ):
            result.append( self.cards.pop() ) 
        return result 

In [17]:
def score( hand ):
    # must be a 5 card hand to be valid 
    if len( hand ) == 5:
        
        # first, look for cards that add up to 15 
        
        points = 0
        
        # fifteens  
        for vector_length in [ 2, 3, 4, 5 ]:
            for vector in combinations( hand, vector_length ):
                if sum( [ card.value for card in vector ] ) == 15:
                    points += 2 
                    
        # pairs (not necessary to account for more than pairs)
        for i, j in combinations( hand, 2 ):
            if i.rank == j.rank:
                points += 2 
                
        # runs
        for vector_len in [ 5, 4, 3 ]:
            for vec in combinations( hand, vector_len ):
                vals = [ card.value for card in vec ]
                run = [ n + min( vals ) for n in range( vector_len ) ]
                if sorted( vals ) == run:
                    points += vector_len
                    break 
                      
    return points 

In [18]:
deck = Deck()
deck.shuffle()
for n in range( 5 ):
    hnd = deck.draw(5)
    print(score(hnd), hnd)

4 [5h, Ac, 8h, 2c, Qs]
7 [Kh, 5c, Qd, 4h, 3d]
2 [6s, Jh, 4d, 10h, 10c]
3 [8d, 9c, Ks, Qc, 2h]
4 [Jd, 5d, 7c, 10s, As]


In [19]:
# # check to see if it's possible to make a score of 19 
# for n in range( 10000 ):
#     deck = Deck()
#     deck.shuffle()
#     hand = deck.draw(5)
#     if score( hand ) == 19:
#         print( 'Uh oh', hand ) 

In [20]:
# all possible hands
all_possible_hands = combinations( deck.cards, 5 ) 

In [55]:
def play_a_hand():
    
    deck = Deck()
    deck.shuffle()
    
    p0_hand = deck.draw(6)
    p1_hand = deck.draw(6)
    turn_card = deck.draw(1)
    crib = []
    
    # player discard 
    crib.append( p0_hand.pop() ) 
    crib.append( p0_hand.pop() ) 
        
    # computer discard 
    crib.append( p1_hand.pop() )
    crib.append( p1_hand.pop() )
        
    #turn card and count hands 
    turn_card = deck.draw(1)
    p0_score = score( p0_hand + turn_card ) 
    p1_score = score( p1_hand + turn_card ) 
    
    # counting
    plays = [] # list of tuples, format: (player, card) 
    for i in range( 4 ):
        plays.append( p0_hand[i] )
        plays.append( p1_hand[i] )
    
#     print( 'Player hand:', p0_hand ) 
#     print( 'Pone hand:', p1_hand ) 
#     print( 'Crib:', crib ) 
#     print( 'Turn:', turn_card ) 
    
    play_vector = list(p0_hand) + list(p1_hand) + crib + list(turn_card) + plays
    return ( p0_score, p1_score ), play_vector 

In [56]:
print( 'Score:', play_a_hand() ) 

Score: ((4, 4), [10h, 9d, 5h, 2h, 9s, 2s, 8d, 6h, 6s, Kc, Qd, 5c, 4c, 10h, 9s, 9d, 2s, 5h, 8d, 2h, 6h])


In [57]:
def play_a_game():
    '''
    Plays a game of cribbage. Returns a winner, 0 or 1
    '''
    
    sc = np.array([0,0]) 
    while True:
        outcome, play_vector = play_a_hand()
        sc += np.array(outcome)
#         print( 'Score:', sc )
        for i, p in enumerate( sc ):
            if p > 120:
                return (i, play_vector) # winner ID 

    return sc

In [58]:
play_a_game()

(1,
 [7d,
  9s,
  2h,
  6d,
  Jh,
  8s,
  Kc,
  9h,
  10h,
  6c,
  10c,
  Qc,
  5c,
  7d,
  Jh,
  9s,
  8s,
  2h,
  Kc,
  6d,
  9h])